In [1]:
from lib.reader import read_alleles_to_num
import numpy as np
import os

from sklearn.model_selection import LeaveOneOut

import argparse

from lib.config import *
import lib.row_cheker as rch

import time

In [2]:
import multiprocessing
import pickle

In [7]:
from importlib import reload

In [5]:
rch = reload(rch)

In [3]:
np.random.seed(239)

parser = argparse.ArgumentParser()
parser.add_argument('-i', type=str, dest='input_table_path')
parser.add_argument('-o', type=str, dest='output_table_path')
parser.add_argument('-n', type=int, dest='rows_number', default=500)
args = parser.parse_args(["-i", "../data_as_npy.npy", "-o", "../res"])

In [4]:
args

Namespace(input_table_path='../data_as_npy.npy', output_table_path='../res', rows_number=500)

In [5]:
table = np.load(args.input_table_path)
print(table.shape)
case_control = np.array([0] * 407 + [1] * 21)

print("Starting on original table...")
#table_2x3 = rch.check_initial_table(table, case_control.astype(bool))
table_2x3 = np.load("../table_2x3.npy")
assert all(table_2x3.sum(1).sum(1) == 428)

(428, 4382202)
Starting on original table...


In [8]:
np.save("../table_2x3", table_2x3)

In [9]:
table_2x3.shape

(4382202, 2, 3)

In [10]:
table = table.astype(int)

In [13]:
loo = LeaveOneOut()

target_rows_list = []
target_pvals_list = []

chi2_hash = pickle.load(open("../chi2_hash_9",  'rb'))

In [14]:
len(chi2_hash)

319738

In [23]:
def do_all_for_single_split(index, table_2x3, case_control, table, chi2_hash, row_number):
    train_index, test_index = index[0], index[1]
    beg = time.time()
    rows, pvals = rch.check_loo_table(table_2x3,
                                      case_control[test_index],
                                      table[test_index].flatten(),
                                      chi2_hash,
                                      False,                                  
                                      cut=args.rows_number)
    end = time.time()
    #print("Done with index {} in {} seconds".format(test_index, end - beg))
    
    return rows, pvals, end - beg

In [59]:
for (train_index, test_index) in train_test_indexes[3:10]:
    beg = time.time()
    rows, pvals = rch.check_loo_table(table_2x3,
                                      case_control[test_index],
                                      table[test_index].flatten(),
                                      chi2_hash,
                                      cut=args.rows_number)
    
    
    
    target_rows_list.append(rows)
    target_pvals_list.append(pvals)
    end = time.time()
    print("Done with index {} in {} seconds".format(test_index, end - beg))
    
    np.save("../1_stage_results/rows/rows_" + str(test_index[0]), rows)
    np.save("../1_stage_results/pvals/pvals_" + str(test_index[0]), pvals)

  0%|          | 315/4382202 [00:00<23:12, 3146.18it/s]

Done with index [3] in 935.8729054927826 seconds


  0%|          | 318/4382202 [00:00<23:02, 3169.33it/s]

Done with index [4] in 972.4523255825043 seconds


  0%|          | 311/4382202 [00:00<23:31, 3104.87it/s]

...

Done with index [7] in 920.5783610343933 seconds


  0%|          | 308/4382202 [00:00<23:43, 3078.76it/s]

Done with index [8] in 902.1052443981171 seconds


100%|██████████| 4382202/4382202 [15:09<00:00, 4816.30it/s]


Done with index [9] in 910.5523960590363 seconds


In [ ]:
target_rows = np.concatenate([x.reshape(1,-1) for x in target_rows_list], 0)
target_pvals = np.concatenate([x.reshape(1,-1) for x in target_pvals_list], 0)

print(target_rows.shape)

np.save(args.output_table_path, target_rows)
np.save(args.output_table_path+"_pvals", target_pvals)

In [27]:
import pickle

In [60]:
len(chi2_hash)

319738

In [44]:
len(chi2_hash)

254387

In [61]:
pickle.dump(chi2_hash, open("../1_stage_results/chi2_hash_9", 'wb'))

In [53]:
np.save("../1_stage_results/rows/rows_1", target_rows_list[1])

In [ ]:
np.save("../1_stage_results/pvals/pvals_1", target_pvals_list[1])

In [15]:
train_test_indexes = [[x,y] for x,y in loo.split(table)]

## test MP

In [ ]:
from multiprocessing import 

In [29]:
pool = multiprocessing.Pool(32)

100%|██████████| 4382202/4382202 [1:29:29<00:00, 816.07it/s]


In [30]:
def g(x):
    return do_all_for_single_split(x, table_2x3, case_control, table, chi2_hash, args.rows_number)

In [45]:
pool.close()

In [ ]:
answ_0_64 = pool.map(g, train_test_indexes[:64])

In [ ]:
answ_64_ = pool.map(g, train_test_indexes[64:])

In [ ]:
answ_10_428

In [151]:
len(answ_0_10)

10

In [ ]:
for i, (r, p, t) in enumerate(answ_64_):
    print(i + 64, t)
    np.save("../1_stage_results/rows/rows_" + str(i + 64), r)
    np.save("../1_stage_results/pvals/pvals_" + str(i + 64), p)

64 2449.892482280731
65 2395.1569764614105
66 2455.095027446747
67 2329.769576072693
68 2394.0045082569122
69 2442.8979539871216
70 2315.367434024811
71 2289.6382389068604
72 2344.7763950824738
73 2327.6323721408844
74 2401.3183856010437
75 2446.5277774333954
76 2297.5950922966003
77 2286.7692651748657
78 2373.7509956359863
79 2340.3492641448975
80 2408.6269178390503
81 2337.512938261032
82 2428.404750585556
83 2409.7876722812653
84 2446.666923522949
85 2444.422253847122
86 2412.7132420539856
87 2474.677628993988
88 2342.659079551697
89 2300.0417976379395
90 2320.7178778648376
91 2470.836761236191
92 2412.415153503418
93 2422.7323808670044
94 2466.7219190597534
95 2398.2065184116364
96 2470.2975363731384
97 2477.3248760700226
98 2428.9790551662445
99 2494.863626718521
100 2480.321272611618
101 2444.629438638687
102 2492.233320236206
103 2496.3695635795593
104 2445.5576469898224
105 2457.790130138397
106 2494.592221736908
107 2479.2668828964233
108 2493.583800792694
109 2472.04647040367

In [157]:
all(answ_0_10[2][0] == target_rows_list[3])

True

In [143]:
target_rows_list[2] == target_rows_list[3]

array([ True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True, False, False,  True, False, False, False,
       False, False, False,  True, False, False, False, False,  True,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True, False,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [ ]:
print("322")

# COLLECT GOOD ROWS

In [5]:
table = np.load(args.input_table_path).astype(int)
print(table.shape)
case_control = np.array([0] * 407 + [1] * 21)

(428, 4382202)


In [6]:
rows_path = "../1_stage_results/rows/"
rows_files = os.listdir(rows_path)

In [7]:
all_rows = []
for f_name in rows_files:
    all_rows.append(np.load(rows_path+f_name))

In [10]:
len(all_rows[0])

500

In [12]:
rows_to_save = set()
for ind in all_rows:
    for row in ind:
        rows_to_save.add(row)

In [20]:
mask = np.array([False] * table.shape[1])
mask[np.array(list(rows_to_save))] = True
print(mask.shape, mask.sum())

(4382202,) 1406


In [24]:
selected_rows = table[:, mask]

In [26]:
head = np.arange(mask.shape[0])[mask]

In [32]:
head

array([  10288,   10612,   16126, ..., 4359974, 4361694, 4366824])

In [34]:
import pandas as pd

In [36]:
result = pd.DataFrame(selected_rows, columns=head)

In [40]:
all(result[10612] == table[:, 10612])

True

In [43]:
for c in result.columns:
    if not all(result[c] == table[:,c]):
        print(c)

In [45]:
result.to_csv("../1_stage_results/result.csv")

In [51]:
np.where(np.array(rows_files) == "rows_0.npy")

(array([217]),)

In [52]:
np.load("../1_stage_results/rows/rows_0.npy") == all_rows[217]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,